In [120]:
import os
import json
import requests
import arweave
import time


# Helper function to mine a given number of blocks
def mine_blocks(n=1, port=1984):
    mine_url = f"http://localhost:{port}/mine/{n}"
    resp = requests.get(mine_url)
    print(f"Mined {n} block(s):", resp.text)

def get_tx_status(txid, port=1984):
    status_url = f"http://localhost:{port}/tx/{txid}/status"
    resp = requests.get(status_url)
    if resp.status_code == 200:
        return resp.json()
    else:
        return None

print("Imports and helper functions ready.")


Imports and helper functions ready.


In [121]:
# === 1. Load Arweave Wallet ===
wallet_file = 'arweave-key.json'
if not os.path.exists(wallet_file):
    raise FileNotFoundError(f"Wallet file '{wallet_file}' not found. Please place your Arweave wallet JSON in the working directory.")

wallet = arweave.Wallet(wallet_file)
print("Loaded wallet with address:", wallet.address)


Loaded wallet with address: s_Li2WJNv45xS0ICR3B5kRQEuDEkXMMgFTsFx094SD4


In [122]:
port = 1984  # Adjust if your ArLocal is on a different port

print("Minting tokens for wallet...")

mint_amount = 100000000000000  # 100k AR in Winston (adjust as needed)
mint_url = f"http://localhost:{port}/mint/{wallet.address}/{mint_amount}"
resp = requests.get(mint_url)
print("Mint response:", resp.text)

# Mine 1 block to process the mint
mine_blocks(1, port=port)

# Check wallet balance
balance_url = f"http://localhost:{port}/wallet/{wallet.address}/balance"
balance_resp = requests.get(balance_url)
if balance_resp.status_code == 200:
    balance_winston = int(balance_resp.text)
    balance_ar = balance_winston / 1e12
    print("Wallet balance (in Winston):", balance_winston)
    print("Wallet balance (in AR):", balance_ar)
else:
    print("Error retrieving wallet balance, status code:", balance_resp.status_code)


Minting tokens for wallet...
Mint response: 100000000000000
Mined 1 block(s): {
  "network": "arlocal.N.1",
  "version": 1,
  "release": 1,
  "queue_length": 0,
  "peers": 1,
  "height": 67,
  "current": "7nf0qecd9j7ij62ixku28m2qksiw70tw5siphbaedptk6g56d7e1t8mim0g56n5v",
  "blocks": 68,
  "node_state_latency": 0
}
Wallet balance (in Winston): 100000000000000
Wallet balance (in AR): 100.0


In [123]:
# === 2. Create, Sign, and Send a Test Transaction ===
sample_data = b"This is a test upload from our decentralized Git server MVP."

# Create the transaction with some tags
tx = arweave.Transaction(wallet, data=sample_data)
tx.add_tag('Content-Type', 'text/plain')
tx.add_tag('RepoID', 'example-repo')
tx.add_tag('CommitHash', 'dummy-commit-hash')

# Sign
tx.sign()
print("Transaction signed.")

# Send
what = tx.send()
print("What:", what)
txid = tx.id
print("Transaction sent! TxID:", tx.id)


Transaction verification failed.

{"data":"VGhpcyBpcyBhIHRlc3QgdXBsb2FkIGZyb20gb3VyIGRlY2VudHJhbGl6ZWQgR2l0IHNlcnZlciBNVlAu","id":"GBSfQB9VBxFiRPYkN08YLvKA1T4ww1pgxc2WrrECN1Q","last_tx":"2bVUmucQlKCpFkh0QMDEopqVkzvzs6X5eWM79xqIT-Cfe2_umNpcgH6aS4g83-M4","owner":"ky-k3Tkp1W04fv027labDNmUOfltkGl6Zf4yBuXPMIEZYGpvV6e_5POd1GS2A34ir-Q8BJJPo_kDEogQKu1E0sofDvmIqxU9w6pWHSnDrvfRVFKqd5HU9Y6NVF4AydhpDoxw28GyI0W42qURA68uKkhzWJUiuZU7ek1LbAN9VoaESaXk9qj3Rp1JvAUWEwYUxQjHUvh2xUO4S0rAtsGjSj8T8it3qMvuPsAQDnuE_EmtEFVh0Osk60OLZaiA1sQj6OjmQ2_UEjuvau_A1RxeNtdKVgvCDhtg5zDveuxbe6-v846E8No0rWMlXQKRWX7dAvmX9iXllrjCarzkC3thm5AvYxkic_9nS6qRrVGibBnEsnhmHMhzvgtoEFkNbig2bbfreQYaCYXkk9waewlZgWpraROl8VHMLMsYjzyAKtkIFNRyKbu1_6v7av9KxdhHldzpNN7gLQKSTlfe_jkOTzn2lJsSFqu2HC4GyiXfTXdf51x-dEwsAAKx3_GyF4LatItODcSTyjAAuBrtxEiTzKjYO49To769tC_N1BJEArStB-2G0OlaDD1C4Fs4sc_DwEPb7n-zTnsMv04WIoUfKP20j8BpKq7fSe2LFhiTzN3auUkDLkV4js0_u2ePr1yPiCw_GioNdU3bWvh8OYcg57aUxOaho1ah2jxOSaqlPd0","quantity":"0","reward":"293857087","signature":"fRHf

Transaction signed.
What: 2bVUmucQlKCpFkh0QMDEopqVkzvzs6X5eWM79xqIT-Cfe2_umNpcgH6aS4g83-M4
Transaction sent! TxID: GBSfQB9VBxFiRPYkN08YLvKA1T4ww1pgxc2WrrECN1Q


In [124]:
# === 3. Check Transaction Status, Then Mine Blocks Until It's Confirmed ===

# We'll do a loop that checks whether the transaction is known and updates on each block.
max_attempts = 5
attempt = 0
is_mined = False

while attempt < max_attempts:
    # Query the transaction status
    status = get_tx_status(txid, port)
    print(f"[Attempt {attempt+1}] Tx status:", status)

    # If the node doesn't return a valid JSON or says not found, we need more blocks or time.
    if not status:
        print("Transaction status not found, possibly not yet indexed. Mining another block...")
        mine_blocks(1, port=port)
        time.sleep(2)
        attempt += 1
        continue

    # If 'status' has "block_height" > 0 or "confirmed" field, that means it's mined
    # (ArLocal's status JSON might differ from mainnet, so adapt if needed).
    if status.get('block_height', 0) > 0 or status.get('confirmed'):
        print("Transaction appears to be mined/confirmed in a block!")
        is_mined = True
        break
    else:
        print("Transaction not yet mined; mining another block...")
        mine_blocks(1, port=port)
        time.sleep(2)
        attempt += 1

if not is_mined:
    print(f"Transaction not mined after {max_attempts} attempts. You can still try to retrieve data, but it may fail.")


[Attempt 1] Tx status: None
Transaction status not found, possibly not yet indexed. Mining another block...
Mined 1 block(s): {
  "network": "arlocal.N.1",
  "version": 1,
  "release": 1,
  "queue_length": 0,
  "peers": 1,
  "height": 68,
  "current": "fwxd07er7mfhv173rro82uomdqa8yvjsasnxo68vf7nwjof96hicrsfp6gbq74un",
  "blocks": 69,
  "node_state_latency": 0
}


[Attempt 2] Tx status: None
Transaction status not found, possibly not yet indexed. Mining another block...
Mined 1 block(s): {
  "network": "arlocal.N.1",
  "version": 1,
  "release": 1,
  "queue_length": 0,
  "peers": 1,
  "height": 69,
  "current": "d5u3gilpykkfhjxh8uc2vntb1t253zu86bwfy62434px5y234mv215z0d1jwgdq0",
  "blocks": 70,
  "node_state_latency": 0
}
[Attempt 3] Tx status: None
Transaction status not found, possibly not yet indexed. Mining another block...
Mined 1 block(s): {
  "network": "arlocal.N.1",
  "version": 1,
  "release": 1,
  "queue_length": 0,
  "peers": 1,
  "height": 70,
  "current": "8bb6itqho2eg2vg95s6tst462owlazn3u001ui9t5wxt3c90hb22kzmhfx498g7k",
  "blocks": 71,
  "node_state_latency": 0
}
[Attempt 4] Tx status: None
Transaction status not found, possibly not yet indexed. Mining another block...
Mined 1 block(s): {
  "network": "arlocal.N.1",
  "version": 1,
  "release": 1,
  "queue_length": 0,
  "peers": 1,
  "height": 71,
  "current": "9luoqifzcmg30a7pmcd

In [125]:
# === 4. Attempt to Retrieve the Transaction Data ===
gateway_url = f"http://localhost:{port}/{txid}"
print("Attempting to fetch data from:", gateway_url)

max_attempts = 5
attempt = 0
retrieved = False

while attempt < max_attempts:
    resp = requests.get(gateway_url)
    if resp.status_code == 200:
        print("Successfully retrieved data from ArLocal:")
        print(resp.content.decode())
        retrieved = True
        break
    else:
        print(f"[Attempt {attempt+1}] 404 or not found. Status code: {resp.status_code}")
        # Mine another block to see if that helps indexing
        mine_blocks(1, port=port)
        time.sleep(2)
    attempt += 1

if not retrieved:
    print("Could not retrieve transaction data after multiple attempts.")


Attempting to fetch data from: http://localhost:1984/GBSfQB9VBxFiRPYkN08YLvKA1T4ww1pgxc2WrrECN1Q
[Attempt 1] 404 or not found. Status code: 404
Mined 1 block(s): {
  "network": "arlocal.N.1",
  "version": 1,
  "release": 1,
  "queue_length": 0,
  "peers": 1,
  "height": 73,
  "current": "vwa2lsjtpej1iqw82h4szvck22ja165b2eafrls8hzf871z9xpxf0wbvmp2l8wup",
  "blocks": 74,
  "node_state_latency": 0
}
[Attempt 2] 404 or not found. Status code: 404
Mined 1 block(s): {
  "network": "arlocal.N.1",
  "version": 1,
  "release": 1,
  "queue_length": 0,
  "peers": 1,
  "height": 74,
  "current": "koq445c6pbn3nwz93oqtt12xcxzc5k2h9bb9yg4ujwsbf23tuuu477dq2ujzzcve",
  "blocks": 75,
  "node_state_latency": 0
}
[Attempt 3] 404 or not found. Status code: 404
Mined 1 block(s): {
  "network": "arlocal.N.1",
  "version": 1,
  "release": 1,
  "queue_length": 0,
  "peers": 1,
  "height": 75,
  "current": "ibszvix35jfoujaovyhc0gg20bucywrn42clepypd5w60m7aajt2b6khyc5u7mcj",
  "blocks": 76,
  "node_state_latency"

In [126]:
# def get_tx(blah):
    # print(f"Fetching transaction data for {tx_id}...")
    # resp = requests.get("http://localhost:1984/wallet/Bj1GnV4PFlHxEoEy9mtjumwRzhaOai9B8hUoY5tE810/balance")
    # https://arweave.net/wallet/{address}/last_tx
    # resp = requests.get(f"http://localhost:1984/wallet/{blah}/last_tx")
    # print(resp.text)
    
# get_tx("Bj1GnV4PFlHxEoEy9mtjumwRzhaOai9B8hUoY5tE810")

arweave.Wallet

arweave.arweave_lib.Wallet